# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [15]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [82]:
def date_sorter():
    
    import pandas as pd

    doc = []
    with open('assets/dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    
    import re
    # 04/20/2009 or 4/3/09 or 12-7-23 or 12-8-89 or 12-4-1986
    pattern1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    
    # Mar 20th, 2009 or Mar 2009 or April. 1990
    pattern2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+.\s][,]{0,1}\d{4})'
    
    # 23 Oct 2021 or 4 May 2001
    pattern3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    
    # Feb 2009, Mar 2011
    pattern4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    
    # 6/1985, 11-2005
    pattern5 = '(\d{1,2}[/-][1|2]\d{3})'
    
    # 1989, 2006
    pattern6 = '([12]\d{3})'
    
    full_regex = '({}|{}|{}|{}|{}|{})'.format(pattern1, pattern2, pattern3, pattern4, pattern5, pattern6)
    modified_date = df.str.extract(full_regex)
    modified_date = modified_date.iloc[:, 0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')
    modified_date = pd.Series(modified_date)
    copied = modified_date.copy()
    modified_date = pd.to_datetime(modified_date)
    
    for i in range(len(modified_date)):
        if int(modified_date[i].year) > 2050:
            #print('h')
            modified_date[i] = modified_date[i].replace(year = modified_date[i].year - 100)
    
    sorted_date = modified_date.sort_values(ascending = True)
    sorted_date = sorted_date.reset_index(drop=False)
    #sorted_date = sorted_date.sort_values(by = [0, 'index'])
    series = sorted_date['index']
    #return series
    
    joined = pd.concat([df, copied, modified_date, sorted_date], axis = 1)
    joined = pd.DataFrame(joined)
    pd.set_option('display.max_rows', None)
    print(joined)
    #return pd.Series(modified_date.values)
date_sorter()

                                                     0                 0  \
0         03/25/93 Total time of visit (in minutes):\n          03/25/93   
1                       6/18/85 Primary Care Doctor:\n           6/18/85   
2    sshe plans to move as of 7/8/71 In-Home Servic...            7/8/71   
3                7 on 9/27/75 Audit C Score Current:\n           9/27/75   
4    2/6/96 sleep studyPain Treatment Pain Level (N...            2/6/96   
5                    .Per 7/06/79 Movement D/O note:\n           7/06/79   
6    4, 5/18/78 Patient's thoughts about current su...           5/18/78   
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...          10/24/89   
8                         3/7/86 SOS-10 Total Score:\n            3/7/86   
9             (4/10/71)Score-1Audit C Score Current:\n           4/10/71   
10   (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...           5/11/85   
11                       4/09/75 SOS-10 Total Score:\n           4/09/75   
12   8/01/98

In [89]:
def date_sorter():
    
    a1_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    a1_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    a1 = pd.concat([a1_1, a1_2])
    a1.reset_index(inplace=True)
    a1_index = a1['level_0']
    
    a2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    a2.reset_index(inplace=True)
    a2_index = a2['level_0']
    
    a3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    a3.reset_index(inplace=True)
    a3_index = a3['level_0']
    
    a6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    a6.reset_index(inplace=True)
    a6_index = a6['level_0']
    save=[]
    for i in a6_index:
        if not(i in a1_index.values):
            save.append(i)
    save = np.asarray(save)
    a6 = a6[a6['level_0'].isin(save)]

    
    a7_1 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    a7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    a7 = pd.concat([a7_1, a7_2])
    a7.reset_index(inplace=True)

    a7_index = a7['level_0']
    save=[]
    for i in a7_index:
        if not((i in a2_index.values) | (i in a3_index.values) | (i in a6_index.values)):
            save.append(i)
    save = np.asarray(save)
    a7 = a7[a7['level_0'].isin(save)]
    
    s = a1.level_0.values.tolist() + a2.level_0.values.tolist() + a3.level_0.values.tolist() + a6.level_0.values.tolist() + a7.level_0.values.tolist()
    s = np.asarray(s)
    
    a1.columns = ['level_0', 'match', 'month', 'day', 'year']
    a1['year'] = a1['year'].apply(str)
    a1['year'] = a1['year'].apply(lambda x: '19' + x if len(x) <= 2 else x)
   
    a2[1] = a2[1].apply(lambda x: x.replace(',', ''))
    a2['day'] = a2[1].apply(lambda x: x.split(' ')[0])
    a2['year'] = a2[1].apply(lambda x: x.split(' ')[1])
    a2.columns = ['level_0', 'match', 'month', 'day-year', 'day', 'year']
    a2.drop('day-year', axis=1, inplace=True) 
    
    a3.columns = ['level_0', 'match', 'day', 'month', 'year']
    a3['day'] = a3['day'].replace(np.nan, -99)
    a3['day'] = a3['day'].apply(lambda x: 1 if int(x) == -99 else x)

    a3['month'] = a3.month.apply(lambda x: x[:3])
    a3['month'] = pd.to_datetime(a3.month, format='%b').dt.month
    
    a6.columns = ['level_0', 'match', 'month', 'year']
    a6['day'] = 1
  
    a7.columns = ['level_0', 'match', 'year']
    a7['day'] = 1
    a7['month'] = 1
   
    final = pd.concat([a1, a2, a3, a6, a7])
    final['date'] = pd.to_datetime(final['month'].apply(str) + '/' + final['day'].apply(str) + '/' + final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')

    myList = final['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(myList), key=lambda x: x[1])], np.arange(500))
    return answer

date_sorter()


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46      49
47     317
48      11
49     319
50      40
51     418
52     165
53     370
54     382
55       3
56      50
57     363
58     219
59     465
60     237
61      23
62     342
63     204
64     258
65     315
66      27
67      93
68      17
69     303
70     488
71     283
72     395
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     318
82     369
83     493
84     239
85     148
86     105
87     336
88       6
89     200
90      81

In [90]:
import numpy as np
s_test = date_sorter()

def run_df_modified_check():
    """
    Check if df appears to be modified.
    """
    try:
        assert type(df) == pd.Series
        assert (df.index == pd.RangeIndex(start=0, stop=500, step=1)).all()
        assert (df.apply(type) == str).all()
        assert df.str.len().min() >= 6
        assert df.str[5].apply(ord).sum() == 38354
        print("Passed df modification check")
    except:
        print("Failed df modification check")

run_df_modified_check()

# check if running the code twice produces the same result
try:
    assert (date_sorter() == s_test).all()
    print("Passed repeatability check")
except:
    print("Failed repeatability check")

# check if the result has the expected index
try:
    assert type(date_sorter().index) == pd.RangeIndex
    assert (date_sorter().index == pd.RangeIndex(start=0, stop=500, step=1)).all()
    print("Passed index check")
except:
    print("Failed index check")

# check the tie-break sort for a sample of records where some have the same date
# note that this only tests a sample and does not check the entire answer
try:
    i_test = [s_test.index[s_test == v].values[0]
              for v in [318, 369, 493, 252, 314, 410, 490]]
    assert sorted(i_test) == i_test
    print("Passed secondary sort sample check")
except:
    print("Failed secondary sort sample check")

def run_v_check(s_test):
    """
    Check if the parsed dates appear to be correct and correctly sorted.
    The check works by producing some test checksums
    if you get for example a False entry in the agree column for
    index value 20 that would mean you have at least one incorrectly
    parsed or incorrectly sorted date in the **output** index
    range 20,21,...,29
    The results of the test are printed.
    Args:
    s_test: Series such as produced by date_sorter()
    Returns:
    None
    """
    try:
        v_check = pd.DataFrame({'correct':
        [6695, 14428, 16742, 9275, 12290, 14654, 9421, 10185, 11464, 16491,
         11797, 14036, 15459, 9412, 13069, 10400, 10498, 14322, 13274, 11001,
         11383, 11910, 10977, 9692, 10199, 10187, 15456, 13491, 9186, 13646,
         11142, 13724, 10994, 12905, 15968, 16648, 13966, 14607, 16932, 14622,
         17942, 18220, 17818, 18305, 19633, 12522, 13978, 18445, 20156, 14797],
        'learner':[
        (s_test.iloc[10*i:(i+1)*10].values * np.array(range(1,11))).sum() for i in range(50)]},
        index=range(0,500,10)).assign(agree=lambda x:x['correct']==x['learner'])
        print("Values checksums:")
        print(v_check)
        assert v_check['agree'].all()
        print("Passed values check")
    except:
        print("Failed values check")
    return

run_v_check(s_test)

Passed df modification check
Passed repeatability check
Failed index check
Passed secondary sort sample check
Values checksums:
     correct  learner  agree
0       6695     6695   True
10     14428    14428   True
20     16742    16742   True
30      9275     9275   True
40     12290    12290   True
50     14654    14654   True
60      9421     9421   True
70     10185    10185   True
80     11464    11464   True
90     16491    16491   True
100    11797    11797   True
110    14036    14036   True
120    15459    15459   True
130     9412     9412   True
140    13069    13069   True
150    10400    10400   True
160    10498    10498   True
170    14322    14322   True
180    13274    13274   True
190    11001    11001   True
200    11383    11383   True
210    11910    11910   True
220    10977    10977   True
230     9692     9692   True
240    10199    10199   True
250    10187    10187   True
260    15456    15456   True
270    13491    13491   True
280     9186     9186   True
29